<a href="https://colab.research.google.com/github/dothangTH/no2_prediction/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [244]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import math
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Define path to the data containing folder

In [245]:
DIR = '/content/drive/MyDrive/CVDHD_data/'

#Train *1st* model
1st model use to predict data without temporal_NO2 feature

Load train.csv, drop NaN values

In [246]:
train_df = pd.read_csv(DIR + 'train.csv')
train_df.dropna(subset=['NO2', 'population_dens'], inplace=True)

Filter outliers

In [247]:
q99 = np.quantile(train_df['NO2'].values, 0.99)
q01 = np.quantile(train_df['NO2'].values, 0.01)
train_df.drop(train_df[train_df['NO2'] >= q99].index, inplace=True)
train_df.drop(train_df[train_df['NO2'] <= q01].index, inplace=True)

Fill satellite_NO2 NaN value with median value (not used in model)

In [248]:
# is_nan = train_df.isna()['satellite_NO2']

# for name in train_df['name'].unique():
#     for index, row in train_df.loc[train_df['name'] == name].iterrows():
#         if is_nan[index]:
#             train_df.loc[index, 'satellite_NO2'] = train_df.loc[train_df['name'] == name]['satellite_NO2'].median(skipna=True)

Calculate temporal_NO2 used for 2nd model training

In [249]:
train_df = train_df.reset_index(drop=True)

for name in train_df['name'].unique():
    first_idx = train_df.loc[train_df['name'] == name].head().index[0]
    for index, row in train_df.loc[train_df['name'] == name].iterrows():
        if index == first_idx:
            train_df.loc[index, 'NO2_temporal'] = row['NO2']
        else:
            numerator = 0
            denominator = 0
            for i in range(max(-3, first_idx - index), 0):
                numerator += train_df.loc[index + i]['NO2'] * (1 / i**2)
                denominator += 1 / i**2
            train_df.loc[index, 'NO2_temporal'] = round(numerator / denominator, 10)

Defined X_train, y_train

In [250]:
y_train = train_df['NO2']
X_train = train_df.drop(['NO2', 'time','lon', 'lat', 'name', 'satellite_NO2', 'NO2_temporal'], 1)

Scaling data

In [251]:
scaler_1st = StandardScaler()
scaler_1st.fit(X_train)
X_train = pd.DataFrame(scaler_1st.transform(X_train), columns=X_train.columns)

Train 1st model

In [252]:
num_folds = 4
kfold = KFold(n_splits=num_folds, shuffle=True)


fold_idx = 1

for train_ids, val_ids in kfold.split(X_train, y_train):

  model_1st = SVR(kernel='rbf', C=10, epsilon=0.1, gamma='scale')
  
  print("Bắt đầu train Fold ", fold_idx)

  # Train model
  model_1st.fit(X_train.iloc[train_ids], y_train.iloc[train_ids])

  # Test và in kết quả
  y_pred = model_1st.predict(X_train.iloc[val_ids])
  y_test = y_train.iloc[val_ids].to_numpy()

  scores = (mean_squared_error(y_test, y_pred, squared=False))

  print("Score:", scores)
  # Sang Fold tiếp theo
  fold_idx = fold_idx + 1

Bắt đầu train Fold  1
Score: 9.825786924200221
Bắt đầu train Fold  2
Score: 9.74033056047768
Bắt đầu train Fold  3
Score: 9.81119679789094
Bắt đầu train Fold  4
Score: 9.968274034935948


#Train *2nd* model

Define X_train, y_train

In [253]:
y_train = train_df['NO2']
X_train = train_df.drop(['NO2', 'time','lon', 'lat', 'name', 'satellite_NO2'], 1)

scaling data

In [254]:
scaler_2nd = StandardScaler()
scaler_2nd.fit(X_train)
X_train = pd.DataFrame(scaler_2nd.transform(X_train), columns=X_train.columns)

Train 2nd model

In [255]:
num_folds = 4
kfold = KFold(n_splits=num_folds, shuffle=True)


fold_idx = 1

for train_ids, val_ids in kfold.split(X_train, y_train):

  model_2nd = SVR(kernel='rbf', C=10, epsilon=0.1, gamma='scale')
  
  print("Bắt đầu train Fold ", fold_idx)

  # Train model
  model_2nd.fit(X_train.iloc[train_ids], y_train.iloc[train_ids])

  # Test và in kết quả
  y_pred = model_2nd.predict(X_train.iloc[val_ids])
  y_test = y_train.iloc[val_ids].to_numpy()

  scores = (mean_squared_error(y_test, y_pred, squared=False))

  print("Score:", scores)
  # Sang Fold tiếp theo
  fold_idx = fold_idx + 1

Bắt đầu train Fold  1
Score: 6.925644981122268
Bắt đầu train Fold  2
Score: 6.979116134934777
Bắt đầu train Fold  3
Score: 6.966975453395968
Bắt đầu train Fold  4
Score: 6.739214675324632


Plot function: Use to plot correlation between *feature* and *NO2*

In [256]:
def plot_pair_grid(feature, name = None, X=train_df):
    if name is not None:
      fig, ax = plt.subplots(figsize=(16, 8 ))
      ax.set_xlabel(feature)
      ax.set_ylabel('NO2')
      plt.title('{} - NO2 correlation (Station: {})'.format(feature, name), color='r')
      plt.scatter(x = X.loc[X['name'] == name][feature], y = X.loc[X['name'] == name]['NO2'])
    else:
      fig, ax = plt.subplots(figsize=(16, 8 ))
      ax.set_xlabel(feature)
      ax.set_ylabel('NO2')
      plt.title('{} - NO2 correlation'.format(feature), color='r')
      plt.scatter(x = X[feature], y = X['NO2'])

#Predict

This code will generate new .csv files in process

In this code we just predict first 4 days of a random month

Define which month to generate data

In [266]:
MONTH = '10'

Predict 1st day (without temporal_NO2)

In [267]:
df = pd.read_csv(DIR + '2019' + MONTH + '01.csv').drop(['Unnamed: 0'], 1)
test_df = pd.DataFrame(data=df)

test_df = test_df[['TEMP', 'RH', 'HPBL', 'PRESSURE',
       'WINDSPEED', 'water', 'agri', 'urban', 'forest',
       'road_dens', 'population_dens', 'dew_point']]

for col in test_df.columns:
  test_df.drop(test_df[test_df[col] == -9999].index, inplace=True)

test_df = test_df.dropna()

for col in test_df.columns:
  test_df.drop(test_df[test_df[col] == -9999].index, inplace=True)

test_df = test_df.dropna()

test_df = pd.DataFrame(scaler_1st.transform(test_df), columns = test_df.columns, index=test_df.index)

predict = model_1st.predict(test_df)
test_df['pred'] = predict

for index, row in test_df.iterrows():
    df.loc[index, 'pred'] = row['pred']

df = df[['lat', 'lon', 'pred']]

df.to_csv('2019' + MONTH + '01.csv')

Predict 2nd, 3rd and 4th day (with temporal_NO2)

In [268]:
DAY = ['02', '03', '04']

for day in DAY:
    prev_dfs = []
    prev_dfs_na = []
    df = pd.read_csv(DIR + '2019' + MONTH + day +'.csv').drop(['Unnamed: 0'], 1)
    test_df = pd.DataFrame(data=df)

    for col in test_df.columns:
      test_df.drop(test_df[test_df[col] == -9999].index, inplace=True)

    test_df = test_df[['TEMP', 'RH', 'HPBL', 'PRESSURE',
          'WINDSPEED', 'water', 'agri', 'urban', 'forest',
          'road_dens', 'population_dens', 'dew_point']]

    for i in range(1, int(day)):
        temp = pd.read_csv('2019' + MONTH + str(i).zfill(2) + '.csv')
        prev_dfs.append(temp)
        prev_dfs_na.append(temp.isna())

    #Calculating temporal_NO2
    for index, row in test_df.iterrows():
        numerator = 0
        denominator = 0
        for i in range(len(prev_dfs)):
            if not prev_dfs_na[i].loc[index, 'pred']:
                denominator += 1 / (len(prev_dfs) - i) ** 2
                numerator += prev_dfs[i].loc[index, 'pred'] / (len(prev_dfs) - i) ** 2
            
        try:
            test_df.loc[index, 'temporal_NO2'] = round(numerator / denominator, 10)
        except:
            test_df.loc[index, 'temporal_NO2'] = np.nan



    test_df = test_df.dropna()

    test_df = pd.DataFrame(scaler_2nd.transform(test_df), columns = test_df.columns, index=test_df.index)
    predict = model_2nd.predict(test_df)
    test_df['pred'] = predict

    for index, row in test_df.iterrows():
        df.loc[index, 'pred'] = row['pred']
        
    df = df[['lat', 'lon', 'pred']]

    df.to_csv('2019' + MONTH + day + '.csv')
    print('day: {}    month: {}'.format(day, MONTH))

day: 02    month: 10
day: 03    month: 10
day: 04    month: 10
